# Request

In [1]:
# import re
# import requests
# from bs4 import BeautifulSoup

In [66]:
# def geocode_address(
#     address: str
# ) -> str:
#     """
#     Generate latitude and longitude from address using Google Maps.
#     Args:
#         address: Address to be geocoded.
#     """
#     # User agent and headers from: https://stackoverflow.com/a/51955893
#     user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14'
#     headers = {'User-Agent': user_agent,'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

#     address = address.replace(' ', '+')
#     r = requests.get(f'https://www.google.com.br/maps/place/{address}',headers=headers)
#     soup = BeautifulSoup(r.content, 'html.parser')
#     try:
#         full_content = ''
#         for item in soup.find_all('meta'):
#             full_content += item['content']

#         latlong = re.findall(r'-\d+.\d+,-\d+.\d+', full_content)[0]

#         time.sleep(2)    
#         return latlong
#     except:
#         return None

In [22]:
# # Bing
# user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14'
# headers = {'User-Agent': user_agent,'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

# address = 'RODOVIA 163 - KM 118 - ZONA RURAL. CEP:79950-000 - NAVIRAI/MS'
# r = requests.get(f'https://www.bing.com/maps?q={address}',headers=headers)
# soup = BeautifulSoup(r.content, 'html.parser')

# print(soup.prettify())

# Selenium

In [1]:
import os
import re
import time
import pandas as pd
import selenium.webdriver as webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options

user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/114.0'
mozilla_driver = os.path.join('../data/01_raw', 'geckodriver.exe')
mozilla_service = Service(mozilla_driver, log_output='../logs/geckodriver.log')
mozilla_options = Options()
mozilla_options.set_preference('general.useragent.override', user_agent)

In [18]:
rf_renovabio_plants = catalog.load("refined_renovabio_plants").reset_index(drop=True)

[08/10/23 22:31:24] INFO     Loading data from 'refined_renovabio_plants' (ParquetDataSet)...   ]8;id=75476;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=175736;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

In [19]:
rf_renovabio_plants = rf_renovabio_plants.fillna('')
address_list = (
    rf_renovabio_plants['DS_END']
    .str.cat(
        [rf_renovabio_plants['NO_END'],
        rf_renovabio_plants['CIDADE'],
        rf_renovabio_plants['UF'],
        rf_renovabio_plants['CEP']],
        sep=' '
    )
).tolist()

address_list2 = (
    rf_renovabio_plants['CIDADE']
    .str.cat(
        [rf_renovabio_plants['UF'],
        rf_renovabio_plants['CEP']],
        sep=' '
    )
).tolist()

In [20]:
def get_latlong_from_url(url: str) -> list:
    """
    Returns latitude and longitude from a Bing Maps URL
    """
    latlong = re.findall(r'-?\d{1,2}\.\d{3,}', url)
    latitude = float(latlong[0])
    longitude = float(latlong[1])
    return latitude, longitude

# Launch browser
browser = webdriver.Firefox(service=mozilla_service, options=mozilla_options)

# Geocoding addresses
latitude = []
longitude = []

first_iter = True
for i in range(len(address_list)):
    browser.get(f'https://www.bing.com/maps?q={address_list[i]}')

    # Longer sleep time for first iteration (browser opens)
    if first_iter is True:
        time.sleep(6)
        first_iter = False
    else:
        time.sleep(3.5)

    lat, long = get_latlong_from_url(browser.current_url)

    # If address is not found, try to geocode only the last 3 address columns
    ckeck_results = re.findall('Não há resultados para:', browser.page_source)
    if len(ckeck_results) > 0:
        browser.get(f'https://www.bing.com/maps?q={address_list2[i]}')
        time.sleep(3)

        lat, long = get_latlong_from_url(browser.current_url)

    latitude.append(lat)
    longitude.append(long)            

browser.quit()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:28                                                                                   │
│                                                                                                  │
│   25 │   else:                                                                                   │
│   26 │   │   time.sleep(3.5)                                                                     │
│   27 │                                                                                           │
│ ❱ 28 │   lat, long = get_latlong_from_url(browser.current_url)                                   │
│   29 │                                                                                           │
│   30 │   # If address is not found, try to geocode only the last 3 words                         │
│   31 │   ckeck_results = re.findall('Não há resultados para:', browser.page_source)              │
│                                                                                                  │
│ in get_latlong_from_url:6                                                                        │
│                                                                                                  │
│    3 │   Returns latitude and longitude from a Bing Maps URL                                     │
│    4 │   """                                                                                     │
│    5 │   latlong = re.findall(r'-?\d{1,2}\.\d{3,}', url)                                         │
│ ❱  6 │   latitude = float(latlong[0])                                                            │
│    7 │   longitude = float(latlong[1])                                                           │
│    8 │   return latitude, longitude                                                              │
│    9                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: list index out of range

In [21]:
df_latlong = pd.DataFrame({'LATITUDE': latitude, 'LONGITUDE': longitude})
new_df = pd.concat([rf_renovabio_plants, df_latlong], axis=1, ignore_index=False)

In [22]:
new_df

,CNPJ,RAZAO_SOCIAL,PROCESSO_CERTIFICACAO,CIDADE,UF,ENDERECO_PRIMARIO,CEP,DS_END,NO_END,BIOCOMBUSTIVEL,...,CD_ROTA,LITROS_CBIO,PCT_VOLUME_ELEGIVEL,NOTA_EF_ENERGETICO_AMBIENTAL,FATOR_EMISSAO_CBIO,INSPETORA,DT_APROVACAO_ANP,DT_VALIDADE,LATITUDE,LONGITUDE
0,44836856000177,AGROINDUSTRIAL VISTA ALEGRE S. A. EM RECUPERAC...,48610206100202048,ITAPETININGA,SP,"FAZENDA VISTA ALEGRE S/N PINHAL, ITAPETININGA ...",18209600,FAZENDA VISTA ALEGRE PINHAL,SN,ETANOL HIDRATADO,...,4,934.479878,93.90,53.40,0.001070,BENRI CLASSIFICACAO DA PRODUCAO DE ACUCAR E ET...,2020-09-16,2023-09-15,-23.423062,-47.951714
1,44836856000177,AGROINDUSTRIAL VISTA ALEGRE S. A. EM RECUPERAC...,48610206100202048,ITAPETININGA,SP,"FAZENDA VISTA ALEGRE S/N PINHAL, ITAPETININGA ...",18209600,FAZENDA VISTA ALEGRE PINHAL,SN,ETANOL ANIDRO,...,4,885.530171,93.90,53.80,0.001129,BENRI CLASSIFICACAO DA PRODUCAO DE ACUCAR E ET...,2020-09-16,2023-09-15,-23.423062,-47.951714
2,00738822000255,SANTA CRUZ ACUCAR E ALCOOL LTDA.,48610200428202051,SANTA CRUZ CABRALIA,BA,FAZENDA SANTA CLARA S/N - CEP: 45.807-000 - SA...,45807000,FAZENDA SANTA CLARA,SN,ETANOL HIDRATADO,...,4,877.476238,100.00,53.40,0.001140,SGS ICS CERTIFICADORA LTDA.,2020-09-16,2023-09-15,-16.380821,-39.438478
3,00738822000255,SANTA CRUZ ACUCAR E ALCOOL LTDA.,48610200428202051,SANTA CRUZ CABRALIA,BA,FAZENDA SANTA CLARA S/N - CEP: 45.807-000 - SA...,45807000,FAZENDA SANTA CLARA,SN,ETANOL ANIDRO,...,4,831.513013,100.00,53.80,0.001203,SGS ICS CERTIFICADORA LTDA.,2020-09-16,2023-09-15,-16.380821,-39.438478
4,28620879000193,IMPACTO BIOENERGIA ALAGOAS S.A.,48610219471201956,TEOTONIO VILELA,AL,"FAZENDA SAO MATHEUS, S/N, ZONA RURAL, 57265-00...",57265000,FAZENDA SAO MATHEUS,SN,ETANOL HIDRATADO,...,4,1122.954412,86.75,48.10,0.000891,INSTITUTO TOTUM DE DESENVOLVIMENTO E GESTAO EM...,2020-09-24,2023-09-23,-9.918905,-36.334803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,05158542000100,CENTRAL ACUCAREIRA USINA SANTA MARIA S/A.,48610217724202333,ZONA RURAL PORTO CALVO,AL,"FAZENDA ENGENHO NOVO, S/N. ZONA RURAL. PORTO C...",57900000,FAZENDA ENGENHO NOVO,SN,ETANOL HIDRATADO,...,4,2692.056710,29.17,59.67,0.000371,BENRI CLASSIFICACAO DA PRODUCAO DE ACUCAR E ET...,2023-07-25,2026-07-24,NaN,NaN
502,05158542000100,CENTRAL ACUCAREIRA USINA SANTA MARIA S/A.,48610217724202333,ZONA RURAL PORTO CALVO,AL,"FAZENDA ENGENHO NOVO, S/N. ZONA RURAL. PORTO C...",57900000,FAZENDA ENGENHO NOVO,SN,ETANOL ANIDRO,...,4,2555.164729,29.17,60.02,0.000391,BENRI CLASSIFICACAO DA PRODUCAO DE ACUCAR E ET...,2023-07-25,2026-07-24,NaN,NaN
503,09094632000217,USINA MONTE ALEGRE AS,48610210276202347,MAMAGUAMPE,PB,"FAZENDA MONTE ALEGRE , S/N ZONA RUARAL - MAMAG...",58280000,FAZENDA MONTE ALEGRE,SN,ETANOL HIDRATADO,...,4,1190.183037,72.88,54.02,0.000840,GREEN DOMUS DESENVOLVIMENTO SUSTENTAVEL LTDA EPP,2023-07-25,2026-07-24,NaN,NaN
504,09094632000217,USINA MONTE ALEGRE AS,48610210276202347,MAMAGUAMPE,PB,"FAZENDA MONTE ALEGRE , S/N ZONA RUARAL - MAMAG...",58280000,FAZENDA MONTE ALEGRE,SN,ETANOL ANIDRO,...,4,1128.973224,72.88,54.37,0.000886,GREEN DOMUS DESENVOLVIMENTO SUSTENTAVEL LTDA EPP,2023-07-25,2026-07-24,NaN,NaN
